In [63]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft
from scipy.signal import find_peaks
import ast
import matplotlib.pyplot as plt

In [52]:
df = pd.read_csv(r'C:\Users\rodge\OneDrive\Desktop\SML\Group Project\comp90051\data\rawData\processedData\waveform_file_names.csv')

In [73]:
ART = df[df['mg_name'] == 'GE_ART'].reset_index(drop=True)
ART.loc[:,'binwave'] = ART['binwave'].apply(lambda x: np.array(ast.literal_eval(x)))

def statistical_ART(signal): 
    return {
        'mean'  :np.mean(signal),
        'std'   :np.std(signal),
        'max'   :np.max(signal),
        'min'   :np.min(signal),
        'range' :np.ptp(signal),
        'skewness':skew(signal),
        'kurtosis':kurtosis(signal),
        'variance':np.var(signal)
    }

ART_matrix = np.array([list(statistical_ART(row).values()) for row in ART['binwave']])
U, S, Vt = np.linalg.svd(ART_matrix, full_matrices=False)
print('Singular Values:', S)


Singular Values: [2.72547916e+06 6.51869409e+03 5.93300480e+02 2.68054609e+02
 5.76830155e+01 7.90109815e+00 4.49263652e-01 5.73106049e-12]


In [84]:
ECG = df[df['mg_name'] == 'GE_ECG'].reset_index(drop=True)
ECG.loc[:,'binwave'] = ECG['binwave'].apply(lambda x: np.array(ast.literal_eval(x)))

def featureExtraction_fft(signal, rate_hz):
    
    fft_transfer = fft(signal)
    magnitude = np.abs(fft_transfer)
    phase = np.angle(fft_transfer)

    freq_resolution = rate_hz / len(signal)

    low_freq_range = (0.5, 10)
    qrs_freq_range = (10, 50)
    high_freq_range = (50, rate_hz/2)

    low_freq_indices = np.floor(np.array(low_freq_range) / freq_resolution).astype(int)
    qrs_freq_indices = np.floor(np.array(qrs_freq_range) / freq_resolution).astype(int)
    high_freq_indices = np.floor(np.array(high_freq_range) / freq_resolution).astype(int)

    low_freq_power = np.sum(magnitude[low_freq_indices[0]:low_freq_indices[1]])
    qrs_freq_power = np.sum(magnitude[qrs_freq_indices[0]:qrs_freq_indices[1]])
    high_freq_power = np.sum(magnitude[high_freq_indices[0]:high_freq_indices[1]])
    peak_freq = np.argmax(magnitude[1:]) * freq_resolution  # Frequency with highest magnitude

    peaks, _ = find_peaks(magnitude)
    num_peaks = len(peaks)

    return {
        'low_freq_power': low_freq_power,
        'qrs_freq_power': qrs_freq_power,
        'high_freq_power': high_freq_power,
        'peak_freq': peak_freq,
        'num_peaks': num_peaks,
        'mean_magnitude': np.mean(magnitude),
        'std_magnitude': np.std(magnitude),
        'mean_phase': np.mean(phase),
        'std_phase': np.std(phase),
        'skewness_magnitude': skew(magnitude),
        'kurtosis_magnitude': kurtosis(magnitude)
    }


feature_names = ['low_freq_power', 'qrs_freq_power', 'high_freq_power', 'peak_freq', 'num_peaks', 
                 'mean_magnitude', 'std_magnitude', 'mean_phase', 'std_phase', 
                 'skewness_magnitude', 'kurtosis_magnitude']

fft_result = []

for i in range(len(ECG)):
    signal = ECG['binwave'][i]
    rate_hz = ECG['hz'][i]
    feature_result = featureExtraction_fft(signal, rate_hz)
    fft_result.append([feature_result[name] for name in feature_names])


ECG_df = pd.DataFrame(fft_result, columns=feature_names)
ECG_matrix = np.array(fft_result)
U, S, Vt = np.linalg.svd(ECG_matrix, full_matrices=False)
print('Singular Values:', S)








Singular Values: [1.58660426e+06 7.34820121e+05 1.36456329e+05 2.34538036e+04
 1.35926601e+03 7.41792669e+02 2.17705371e+02 1.36434765e+02
 1.57793027e+01 1.39031110e+00 2.97851762e-01]
